Original reference : https://github.com/aymericdamien/TensorFlow-Examples <br/>
Modified by : Nazmi Asri

In [1]:
import os
import tensorflow as tf
import numpy as np

from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# Get current directory
CURR_DIR = os.getcwd()

# Get MNIST 
# One-hot encoding is used for categorical variables, as binary vector
mnist = input_data.read_data_sets(CURR_DIR + "/data/", one_hot=True)

Extracting /Users/user/Desktop/Work (AirAsia)/superdatascience/Experiments/data/train-images-idx3-ubyte.gz
Extracting /Users/user/Desktop/Work (AirAsia)/superdatascience/Experiments/data/train-labels-idx1-ubyte.gz
Extracting /Users/user/Desktop/Work (AirAsia)/superdatascience/Experiments/data/t10k-images-idx3-ubyte.gz
Extracting /Users/user/Desktop/Work (AirAsia)/superdatascience/Experiments/data/t10k-labels-idx1-ubyte.gz


In [3]:
# Parameters
learning_rate = 0.01
training_epochs = 25
batch_size = 100
display_epoch = 1
logs_path = '/tmp/tensorflow_logs/example/'

In [4]:
# tf Graph Input
# mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')

# 0-9 digits recognition => 10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

In [5]:
# Construct model and encapsulating all ops into scopes, making
# Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(-tf.reduce_sum(y * tf.log(pred), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

In [6]:
# Initializing the variables
init = tf.global_variables_initializer()

In [7]:
# Create a summary to monitor cost tensor
tf.summary.scalar("loss", cost)

# Create a summary to monitor accuracy tensor
tf.summary.scalar("accuracy", acc)

# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

In [8]:
# Start Training
with tf.Session() as sess:
    sess.run(init)

    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples / batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_epoch == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", "{:.9f}".format(avg_cost))

    print("\n\nOptimization Finished!")

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch: 0001 cost= 1.183846918
Epoch: 0002 cost= 0.665370069
Epoch: 0003 cost= 0.552812204
Epoch: 0004 cost= 0.498645033
Epoch: 0005 cost= 0.465506919
Epoch: 0006 cost= 0.442554539
Epoch: 0007 cost= 0.425598981
Epoch: 0008 cost= 0.412182444
Epoch: 0009 cost= 0.401443272
Epoch: 0010 cost= 0.392368469
Epoch: 0011 cost= 0.384786371
Epoch: 0012 cost= 0.378183731
Epoch: 0013 cost= 0.372381484
Epoch: 0014 cost= 0.367309807
Epoch: 0015 cost= 0.362733021
Epoch: 0016 cost= 0.358568659
Epoch: 0017 cost= 0.354909457
Epoch: 0018 cost= 0.351433035
Epoch: 0019 cost= 0.348328369
Epoch: 0020 cost= 0.345420789
Epoch: 0021 cost= 0.342738916
Epoch: 0022 cost= 0.340275070
Epoch: 0023 cost= 0.337939992
Epoch: 0024 cost= 0.335769073
Epoch: 0025 cost= 0.333708791


Optimization Finished!
Accuracy: 0.9135


Run the command line: tensorboard --logdir=/tmp/tensorflow_logs .
Then open http://0.0.0.0:6006/ into your web browser